In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from modellib.attention import Attention
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold

np.random.seed(42)
tf.random.set_seed(42)

plt.rcParams.update({'font.size': 16})
plt.rcParams["font.weight"] = "bold"
plt.rcParams["axes.labelweight"] = "bold"

2023-04-26 12:40:45.446484: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-26 12:40:45.485242: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-26 12:40:45.485941: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-26 12:40:46.090521: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
with open('./logs/data/dataset.pickle', 'rb') as handle:
    dataset = pickle.load(handle)

In [3]:
def ATT(input_shape):
    inp=tf.keras.layers.Input(input_shape)
    net=Attention((None,*input_shape))(inp)
    out=tf.keras.layers.Dense(units=256,activation="relu")(net)
    att_model=tf.keras.Model(inputs=inp,outputs=out,name="ATT")
    return att_model

def CNN(input_shape):
    cnn_model=\
    tf.keras.Sequential([
        tf.keras.layers.Conv2D(64,(5,5),(2,2),"same",activation="relu",input_shape=input_shape),
        tf.keras.layers.Conv2D(128,(3,3),(2,2),"same",activation="relu"),
        tf.keras.layers.Conv2D(256,(3,3),(2,2),"same",activation="relu"),
        tf.keras.layers.GlobalAveragePooling2D(),
    ],name='CNN')
    return cnn_model

def ANN(input_shape):
    ann_model=\
    tf.keras.Sequential([
        tf.keras.layers.Dense(units=256,activation="relu",input_shape=input_shape),
        tf.keras.layers.Dense(units=64,activation="relu"),
        tf.keras.layers.Dense(units=1,activation="linear")
    ],name='ANN')
    return ann_model

In [4]:
#FUSED MODEL

In [5]:
def get_fused_model():
    X_seq=tf.keras.layers.Input(shape=(10,21))
    X_img=tf.keras.layers.Input(shape=(64,64,3))

    cnn=CNN((64,64,3))
    att=ATT((10,21))
    ann=ANN((512,))

    cnn_emb=cnn(X_img)
    att_emb=att(X_seq)
    net=tf.concat([cnn_emb,att_emb],axis=1)
    out=ann(net)

    fused_model=tf.keras.Model(inputs=[X_seq,X_img],outputs=out,name='FUSED')
    fused_model.compile(loss="mse",optimizer="adam",metrics=["mae","mse"])
    return fused_model

In [6]:
smax=np.max(dataset["seq"],axis=0)
smin=np.min(dataset["seq"],axis=0)
seq_scaled=np.float32((dataset["seq"]-smin)/(smax-smin))
img_scaled=np.float32(dataset['img']/255.)

speed=np.float32(dataset["spd"].reshape(-1,1))

In [7]:
l=[]
fold=KFold(n_splits=10)
for split,(train_idx,test_idx) in enumerate(fold.split(np.arange(speed.shape[0]))):
    print(f"On split: {split}")
    seq_train=seq_scaled[train_idx]
    img_train=img_scaled[train_idx]
    speed_train=speed[train_idx]
    seq_test=seq_scaled[test_idx]
    img_test=img_scaled[test_idx]
    speed_test=speed[test_idx]

    #Training
    fused_model=get_fused_model()
    best_save=tf.keras.callbacks.ModelCheckpoint(filepath=f"./logs/model/tr_{split}.h5",
                                                 save_weights_only=True,
                                                 monitor='val_mae',mode='max',
                                                 save_best_only=True)
    
    training=fused_model.fit(x=[seq_train,img_train],y=speed_train,batch_size=64,epochs=50,
                             validation_split=0.2,callbacks=[best_save])
    #Evaluate
    pred=fused_model.predict([seq_train,img_train],verbose=0)
    mae_train=mean_absolute_error(speed_train,pred)
    mse_train=mean_squared_error(speed_train,pred)
    r2_train=r2_score(speed_train,pred)

    pred=fused_model.predict([seq_test,img_test],verbose=0)
    mae_test=mean_absolute_error(speed_test,pred)
    mse_test=mean_squared_error(speed_test,pred)
    r2_test=r2_score(speed_test,pred)

    res={'spt':split,'mae_train':mae_train,'mse_train':mse_train,'r2_train':r2_train,
     'mae_test':mae_test,'mse_test':mse_test,'r2_test':r2_test}
    l.append(res)
    
    pd.DataFrame(l).to_csv('./Output/res_10fold2.csv',index=False)

On split: 7
Epoch 1/50


2023-04-26 12:40:47.961077: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 367804416 exceeds 10% of free system memory.


117/117 [==============================] - 14s 110ms/step - loss: 336.6964 - mae: 13.3334 - mse: 336.6964 - val_loss: 161.2447 - val_mae: 9.2098 - val_mse: 161.2447
Epoch 2/50
117/117 [==============================] - 13s 107ms/step - loss: 143.1108 - mae: 8.9395 - mse: 143.1108 - val_loss: 113.7206 - val_mae: 8.1069 - val_mse: 113.7206
Epoch 3/50
117/117 [==============================] - 18s 155ms/step - loss: 112.8244 - mae: 7.8521 - mse: 112.8244 - val_loss: 89.1658 - val_mae: 7.0737 - val_mse: 89.1658
Epoch 4/50
117/117 [==============================] - 17s 149ms/step - loss: 93.2539 - mae: 7.0961 - mse: 93.2539 - val_loss: 73.0971 - val_mae: 6.5061 - val_mse: 73.0971
Epoch 5/50
117/117 [==============================] - 15s 124ms/step - loss: 74.1185 - mae: 6.3353 - mse: 74.1185 - val_loss: 56.0737 - val_mae: 5.4856 - val_mse: 56.0737
Epoch 6/50
117/117 [==============================] - 14s 115ms/step - loss: 58.1680 - mae: 5.6233 - mse: 58.1680 - val_loss: 43.5657 - val_mae: 

Epoch 49/50
117/117 [==============================] - 12s 106ms/step - loss: 12.0819 - mae: 2.5530 - mse: 12.0819 - val_loss: 17.7289 - val_mae: 3.0853 - val_mse: 17.7289
Epoch 50/50
117/117 [==============================] - 12s 107ms/step - loss: 12.7005 - mae: 2.6331 - mse: 12.7005 - val_loss: 17.8081 - val_mae: 3.0725 - val_mse: 17.8081


2023-04-26 12:51:29.842770: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 459767808 exceeds 10% of free system memory.


...

2023-04-26 13:01:59.061629: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 459767808 exceeds 10% of free system memory.


On split: 9
Epoch 1/50


2023-04-26 13:02:03.138915: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 367804416 exceeds 10% of free system memory.


117/117 [==============================] - 14s 108ms/step - loss: 355.8886 - mae: 13.7474 - mse: 355.8886 - val_loss: 156.8026 - val_mae: 9.2058 - val_mse: 156.8026
Epoch 2/50
117/117 [==============================] - 12s 106ms/step - loss: 150.8930 - mae: 9.1836 - mse: 150.8930 - val_loss: 104.8974 - val_mae: 7.7875 - val_mse: 104.8974
Epoch 3/50
117/117 [==============================] - 12s 107ms/step - loss: 118.1704 - mae: 8.0523 - mse: 118.1704 - val_loss: 88.2068 - val_mae: 6.9878 - val_mse: 88.2068
Epoch 4/50
117/117 [==============================] - 12s 107ms/step - loss: 98.6758 - mae: 7.2942 - mse: 98.6758 - val_loss: 73.8989 - val_mae: 6.4197 - val_mse: 73.8989
Epoch 5/50
117/117 [==============================] - 13s 107ms/step - loss: 80.2734 - mae: 6.5745 - mse: 80.2734 - val_loss: 58.1169 - val_mae: 5.4312 - val_mse: 58.1169
Epoch 6/50
117/117 [==============================] - 12s 106ms/step - loss: 62.9221 - mae: 5.8210 - mse: 62.9221 - val_loss: 45.8391 - val_mae: 

Epoch 49/50
117/117 [==============================] - 12s 107ms/step - loss: 11.9345 - mae: 2.5147 - mse: 11.9345 - val_loss: 17.9528 - val_mae: 3.1128 - val_mse: 17.9528
Epoch 50/50
117/117 [==============================] - 12s 107ms/step - loss: 12.0593 - mae: 2.5621 - mse: 12.0593 - val_loss: 17.3295 - val_mae: 3.0432 - val_mse: 17.3295


In [ ]:
#NICE